In [ ]:
import numpy as np
import pandas as pd
import glob
import os
#for dirname, _, filenames in os.walk('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd

In [ ]:
Mild_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*.*')
Moderate_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*.*')
NonDemented_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*.*')
VeryMild_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*.*')


data_OVS=[]
labels_OVS = []

for i in NonDemented_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(0)

for i in VeryMild_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(1)

for i in Mild_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(2)

for i in Moderate_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(3)

In [ ]:
data_OVS = np.array(data_OVS)
labels_OVS = np.array(labels_OVS)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
data_OVS,labels_OVS = sm.fit_resample(data_OVS.reshape(-1, 224 * 224 * 3), labels_OVS)

#data_OVS = data_OVS.reshape(-1, 224, 224, 3)

print(data_OVS.shape, labels_OVS.shape)
print(len(data_OVS))

In [ ]:
import glob
Mild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*.*')
Moderate_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*.*')
NonDemented_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*.*')
VeryMild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*.*')


data_test = []
labels_test  = []

for i in NonDemented_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(0)

for i in VeryMild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(1)

for i in Mild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(2)

for i in Moderate_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),color_mode='rgb')
    image=np.array(image)
    data_test.append(image)
    labels_test.append(3)

In [ ]:
data_OVS.shape

In [ ]:
data_test = np.array(data_test)
labels_test = np.array(labels_test)

data_test = data_test.reshape(1279,3*224*224)

#data_OVS = data_OVS.reshape(5121, 224 * 224 * 3)

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='poly',gamma='auto')

#from sklearn.tree import DecisionTreeClassifier
#Decision tree
#Decision_Tree_classifier = DecisionTreeClassifier(criterion='entropy',max_depth=15, random_state=42)

from sklearn.ensemble import RandomForestClassifier
clf  = RandomForestClassifier(n_estimators=130,max_depth=2)

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

XGB_Classifier=xgb.XGBClassifier()

#Creating an XGBoost classifier
XGB_Classifier = xgb.XGBClassifier(max_depth=2, eval_metric='mlogloss', gamma=0, objective= 'multi:softmax')


In [ ]:
#VotingClassifier
from sklearn.ensemble import VotingClassifier

# group / ensemble of models
estimator = []
estimator.append(('SVC', svc))
estimator.append(('RF', clf))
estimator.append(('XGB', XGB_Classifier))


# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(data_OVS, labels_OVS)

In [ ]:
y2 = vot_hard.predict(data_test)

In [ ]:
from sklearn import metrics
accuracy = metrics.accuracy_score(labels_test,y2)
print("Accuracy: {:.2f}".format(accuracy))

from sklearn.metrics import confusion_matrix, classification_report
SVM_CM=confusion_matrix(labels_test,y2)
print('Confusion Matrix: \n', SVM_CM)
print(classification_report(labels_test,y2))